# Fitting using a windowed average

This small notebook carries out the fitting of the windowed average data used in the main notebook. 

Spectral data from a single subject (_sub0_) is fitted after spectra are averaged in a moving window.  

This somewhat simulates analysis approaches used before. Temporal resolution is lost and time point become correlated complicating further analysis. However, SNR is substantially improved over the raw data.

## Load data

In [5]:
import numpy as np

from fsl_mrs.utils import mrs_io
from fsl_mrs.utils import nifti_mrs_tools as ntools
from fsl_mrs.utils.preproc import nifti_mrs_proc as nproc

data_stim = mrs_io.read_FID('simulated_data/sub0_stim.nii.gz')
data_ctrl = mrs_io.read_FID('simulated_data/sub0_ctrl.nii.gz')

print(data_stim.shape)

(1, 1, 1, 1024, 64)


## Perform windowed averaging

In [22]:
def calc_window_indices(idx, window_sz, max):
    """Function to calculate the indicies inside the window
    
    Implements circular indexing at each end.

    :param idx: Current index
    :type idx: int
    :param window_sz: Number of spectra within window
    :type window_sz: int
    :param max: Maximum size of data dimension
    :type max: int
    :return: List of integer indicies
    :rtype: list
    """
    indices = np.arange(window_sz) - np.floor(window_sz / 2) + idx
    indices[indices >= max] -= max
    indices[indices < 0] += max
    return indices.astype(int).tolist()

def window_avg(data, block_size):
    """Unweighted averaging of spectra in blocks.

    :param data: NIfTI-MRS class object with DIM_DYN dimension to average
    :type data: NIFTI_MRS
    :param block_size: Size of the windowing block
    :type block_size: int
    :return: Averaged data
    :rtype: NIFTI_MRS
    """
    avg_data = []
    for idx in range(data.shape[-1]):
        c_ind = calc_window_indices(idx, block_size, data.shape[-1])
        _, block = ntools.split(data,'DIM_DYN', c_ind)
        avg = nproc.average(block, 'DIM_DYN')
        avg_data.append(ntools.reorder(avg, ['DIM_DYN', None, None]))
    return ntools.merge(avg_data, 'DIM_DYN')

wavg_stim = window_avg(data_stim, 5)
wavg_ctrl = window_avg(data_ctrl, 5)



## Fit data

In [28]:
from fsl_mrs.utils.fitting import fit_FSLModel
from fsl_mrs.utils.misc import parse_metab_groups
import pandas as pd

peaks_comb = [['PCh','GPC'],
              ['Cr','PCr'],
              ['NAA', 'NAAG'],
              ['Glu', 'Gln'],
              ['Glc', 'Tau']]

def fit_spectrum(mrs_obj):
    """Fit a single spectrum in an MRS object.

    Basis must already be loaded

    :param mrs_obj: Data to fit
    :type mrs_obj: MRS
    :return: Results object
    :rtype: FitRes
    """
    fit_args = dict(
        ppmlim=(0.2, 4.2),
        baseline_order=0,
        metab_groups=parse_metab_groups(mrs_obj, 'Mac'))
    res = fit_FSLModel(mrs_obj, **fit_args)
    res.combine(peaks_comb)
    return res

def fit_nifti_obj(data):
    mrs_list = data.mrs(basis_file='basis')
    all_res = []
    for mrs in mrs_list:
        all_res.append(fit_spectrum(mrs).fitResults)

    return pd.concat(all_res, ignore_index=True)


## Save data to csv format

In [29]:
from pathlib import Path
out_dir = Path('windowed_avg_results')
out_dir.mkdir(exist_ok=True)

fit_nifti_obj(wavg_stim).to_csv(out_dir / 'sub0_stim_wavg.csv')
fit_nifti_obj(data_stim).to_csv(out_dir / 'sub0_stim.csv')
fit_nifti_obj(wavg_ctrl).to_csv(out_dir / 'sub0_ctrl_wavg.csv')
fit_nifti_obj(data_ctrl).to_csv(out_dir / 'sub0_ctrl.csv')